In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = r"C:\Users\91944\Desktop\Research\Crime\All datasets\cyber-crimes-from-ncrb-master-data-year-state-and-city-wise-total-number-of-cyber-crimes-committed-in-india.csv"
data = pd.read_csv(file_path)

# Data Preprocessing
# Remove 'Total Cities' entries and filter relevant columns
filtered_data = data[data['city'] != 'Total Cities'].dropna(subset=['state', 'value'])

# Create dummy crime categories (since specific categories aren't in the dataset)
def assign_dummy_crimes(row):
    total = row['value']
    fraud = int(total * 0.6)  # Assume 60% as IPC Fraud
    cheating = total - fraud  # Remainder as IT Act Cheating
    return fraud, cheating

filtered_data[['IPC_Fraud', 'IT_Act_Cheating']] = filtered_data.apply(assign_dummy_crimes, axis=1, result_type='expand')

# Aggregate data by state
state_data = filtered_data.groupby('state')[['IPC_Fraud', 'IT_Act_Cheating']].sum().reset_index()

# Calculate proportions
state_data['Total_Crimes'] = state_data['IPC_Fraud'] + state_data['IT_Act_Cheating']
state_data['IPC_Proportion'] = state_data['IPC_Fraud'] / state_data['Total_Crimes']
state_data['IT_Proportion'] = state_data['IT_Act_Cheating'] / state_data['Total_Crimes']

# Sort data for visualization
state_data = state_data.sort_values('IPC_Proportion', ascending=False)

# Diverging bar chart
plt.figure(figsize=(14, 8))
sns.barplot(x='IPC_Proportion', y='state', data=state_data, color='blue', label='IPC Fraud')
sns.barplot(x='IT_Proportion', y='state', data=state_data, color='red', label='IT Act Cheating', 
            left=state_data['IPC_Proportion'])

plt.title('Proportional Distribution of Fraud vs Cheating by Personation Cases (2022)', pad=20)
plt.xlabel('Proportion of Cases')
plt.ylabel('State')
plt.xlim(0, 1)
plt.axvline(0.5, color='black', linestyle='--')
plt.legend()
plt.tight_layout()
plt.savefig("diverging_barchart_proportions.png", dpi=300)
plt.close()

print("Diverging bar chart saved as 'diverging_barchart_proportions.png'")

Diverging bar chart saved as 'diverging_barchart_proportions.png'
